# ARTIST ANALYSIS: SUCCESS RATES, TIMELINES, METRICS

In [1]:
import pandas as pd
df_artists = pd.read_csv("artists_df.csv")
df_timeline = pd.read_csv("timeline_df.csv")
df_timeline.set_index(df_timeline.columns[0], inplace = True)
df_timeline.rename_axis("release_date", axis = 0, inplace = True)

In [2]:
df_artists.head()

,artists,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,popularity,explicit,available_markets,tempo,loudness,duration_min
0,KAROL G,0.803,0.715,2.0,1.0,0.2980,0.2950,0.000134,0.0574,0.574,0.96,0.0,1.0,3.0,6.0,3.0
1,Juice WRLD,0.808,0.745,10.0,0.0,0.3420,0.1450,0.000000,0.2920,0.829,0.94,1.0,1.0,6.0,2.0,4.0
2,Dan + Shay,0.654,0.630,10.0,1.0,0.0259,0.1530,0.000000,0.1110,0.430,0.90,0.0,1.0,1.0,6.0,3.0
3,PARTYNEXTDOOR,0.430,0.700,6.0,1.0,0.2640,0.0902,0.000000,0.2160,0.731,0.89,1.0,1.0,4.0,6.0,3.0
4,21 Savage,0.585,0.520,5.0,0.0,0.0712,0.1240,0.000070,0.1310,0.129,0.88,1.0,1.0,6.0,2.0,4.0


In [3]:
df_timeline.head()

,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,popularity,explicit,id,name,artists,available_markets,tempo,loudness,duration_min
release_date,,,,,,,,,,,,,,,,,,
1967-05-12,0.533,0.905,2,1,0.0754,0.00876,0.5780,0.0698,0.486,0.72,0,0wJoRiX5K5BxlqZTolB2LD,Purple Haze,Jimi Hendrix,0.974684,3,6,3
1967-05-12,0.507,0.847,1,0,0.0411,0.07510,0.2370,0.3720,0.566,0.62,0,23MrkN7g6Q5U7GLIxNHN1B,Foxey Lady,Jimi Hendrix,0.974684,3,6,3
1967-05-12,0.499,0.495,5,1,0.0501,0.34800,0.0286,0.2000,0.339,0.64,0,5H6Jp0syB5yEPk7SWYdlmk,The Wind Cries Mary,Jimi Hendrix,0.974684,2,1,3
1967-05-12,0.346,0.768,9,1,0.0377,0.00603,0.3800,0.0244,0.532,0.71,0,0NWPxcsf5vdjdiFUI8NgkP,Hey Joe,Jimi Hendrix,0.974684,6,6,4
1967-05-12,0.277,0.479,5,0,0.0495,0.58500,0.0224,0.1600,0.483,0.56,0,3XxBSvDZKH5YvZZjTpn6eR,Red House,Jimi Hendrix,0.974684,1,3,4
